Полубаров Александр РТ5-61б
Лабораторная работа №3 Часть 2

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.impute import MissingIndicator

# Загрузка и первичный анализ данных

In [4]:
data = pd.read_csv('data/dc-wikia-data.csv', sep=",")

In [5]:
data.shape

(6896, 13)

In [6]:
data.dtypes

page_id               int64
name                 object
urlslug              object
ID                   object
ALIGN                object
EYE                  object
HAIR                 object
SEX                  object
GSM                  object
ALIVE                object
APPEARANCES         float64
FIRST APPEARANCE     object
YEAR                float64
dtype: object

In [7]:
data.isnull().sum()

page_id                0
name                   0
urlslug                0
ID                  2013
ALIGN                601
EYE                 3628
HAIR                2274
SEX                  125
GSM                 6832
ALIVE                  3
APPEARANCES          355
FIRST APPEARANCE      69
YEAR                  69
dtype: int64

In [8]:
data.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,YEAR
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,"1939, May",1939.0
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2496.0,"1986, October",1986.0
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,1565.0,"1959, October",1959.0
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,NaN,Living Characters,1316.0,"1987, February",1987.0
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,1237.0,"1940, April",1940.0


In [9]:
total_count = data.shape[0]
print('Всего строк: {}'.format(total_count))

Всего строк: 6896


## 1.2.2. Обработка пропусков в категориальных данных

In [10]:
cat_cols = []
for col in data.columns:
    # Количество пустых значений 
    temp_null_count = data[data[col].isnull()].shape[0]
    dt = str(data[col].dtype)
    if temp_null_count>0 and (dt=='object'):
        cat_cols.append(col)
        temp_perc = round((temp_null_count / total_count) * 100.0, 2)
        print('Колонка {}. Тип данных {}. Количество пустых значений {}, {}%.'.format(col, dt, temp_null_count, temp_perc))

Колонка ID. Тип данных object. Количество пустых значений 2013, 29.19%.
Колонка ALIGN. Тип данных object. Количество пустых значений 601, 8.72%.
Колонка EYE. Тип данных object. Количество пустых значений 3628, 52.61%.
Колонка HAIR. Тип данных object. Количество пустых значений 2274, 32.98%.
Колонка SEX. Тип данных object. Количество пустых значений 125, 1.81%.
Колонка GSM. Тип данных object. Количество пустых значений 6832, 99.07%.
Колонка ALIVE. Тип данных object. Количество пустых значений 3, 0.04%.
Колонка FIRST APPEARANCE. Тип данных object. Количество пустых значений 69, 1.0%.


In [11]:
cat_temp_data = data[['ALIGN']]
cat_temp_data.head()

,ALIGN
0,Good Characters
1,Good Characters
2,Good Characters
3,Good Characters
4,Good Characters


In [12]:
cat_temp_data['ALIGN'].unique()

array(['Good Characters', 'Bad Characters', 'Neutral Characters', nan,
       'Reformed Criminals'], dtype=object)

In [13]:
cat_temp_data[cat_temp_data['ALIGN'].isnull()].shape

(601, 1)

In [14]:
imp2 = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
data_imp2 = imp2.fit_transform(cat_temp_data)
data_imp2

array([['Good Characters'],
       ['Good Characters'],
       ['Good Characters'],
       ...,
       ['Good Characters'],
       ['Good Characters'],
       ['Bad Characters']], dtype=object)

In [15]:
np.unique(data_imp2)

array(['Bad Characters', 'Good Characters', 'Neutral Characters',
       'Reformed Criminals'], dtype=object)

In [16]:
imp3 = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='Non-defined')
data_imp3 = imp3.fit_transform(cat_temp_data)
data_imp3

array([['Good Characters'],
       ['Good Characters'],
       ['Good Characters'],
       ...,
       ['Good Characters'],
       ['Good Characters'],
       ['Bad Characters']], dtype=object)

In [17]:
np.unique(data_imp3)

array(['Bad Characters', 'Good Characters', 'Neutral Characters',
       'Non-defined', 'Reformed Criminals'], dtype=object)

In [18]:
data_imp3[data_imp3=='Non-defined'].size

601

# 2. Преобразование категориальных признаков в числовые

In [19]:
cat_enc = pd.DataFrame({'c1':data_imp2.T[0]})
cat_enc

,c1
0,Good Characters
1,Good Characters
2,Good Characters
3,Good Characters
4,Good Characters
...,...
6891,Good Characters
6892,Good Characters
6893,Good Characters
6894,Good Characters


## 2.1. Кодирование категорий целочисленными значениями - label encoding

In [20]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [21]:
le = LabelEncoder()
cat_enc_le = le.fit_transform(cat_enc['c1'])

In [22]:
cat_enc['c1'].unique()

array(['Good Characters', 'Bad Characters', 'Neutral Characters',
       'Reformed Criminals'], dtype=object)

In [23]:
np.unique(cat_enc_le)

array([0, 1, 2, 3])

In [24]:

le.inverse_transform([0, 1, 2, 3])

array(['Bad Characters', 'Good Characters', 'Neutral Characters',
       'Reformed Criminals'], dtype=object)

## 2.2. Кодирование категорий наборами бинарных значений - one-hot encoding

In [25]:
ohe = OneHotEncoder()
cat_enc_ohe = ohe.fit_transform(cat_enc[['c1']])

In [26]:
cat_enc.shape

(6896, 1)

In [27]:
cat_enc_ohe.shape

(6896, 4)

In [28]:
cat_enc_ohe

<6896x4 sparse matrix of type '<class 'numpy.float64'>'
	with 6896 stored elements in Compressed Sparse Row format>

In [29]:
cat_enc_ohe.todense()[0:10]

matrix([[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [30]:
cat_enc.head(10)

,c1
0,Good Characters
1,Good Characters
2,Good Characters
3,Good Characters
4,Good Characters
5,Good Characters
6,Good Characters
7,Good Characters
8,Good Characters
9,Good Characters


## 2.3. Pandas get_dummies - быстрый вариант one-hot кодирования

In [31]:
pd.get_dummies(cat_enc).head()

,c1_Bad Characters,c1_Good Characters,c1_Neutral Characters,c1_Reformed Criminals
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0


In [32]:
pd.get_dummies(cat_temp_data, dummy_na=True).head()

,ALIGN_Bad Characters,ALIGN_Good Characters,ALIGN_Neutral Characters,ALIGN_Reformed Criminals,ALIGN_nan
0,0,1,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0
